In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
path_train="/content/drive/My Drive/plant-pathology-2020-fgvc7/train.csv"
path_test="/content/drive/My Drive/plant-pathology-2020-fgvc7/test_1.csv"
plant_test=pd.read_csv(path_test)
plant_train=pd.read_csv(path_train)

In [ ]:
import os, random
import shutil

src_dir1 = "/content/drive/My Drive/plant-pathology-2020-fgvc7/images/"

dst_dir1 = "/content/drive/My Drive/plant-pathology-2020-fgvc7/training_2/multiple/"
dst_dir2 = "/content/drive/My Drive/plant-pathology-2020-fgvc7/training_2/rust/"
dst_dir3 = "/content/drive/My Drive/plant-pathology-2020-fgvc7/training_2/scab/"
dst_dir4 = "/content/drive/My Drive/plant-pathology-2020-fgvc7/training_2/t_none/"

#file_list1 = os.listdir(src_dir1)
#file_list2 = os.listdir(src_dir2)
img_id = plant_train.loc[ : , 'image_id' ]
img_name = img_id.values.tolist()
mul = plant_train.loc[ : , 'multiple_diseases' ]
mul_val= mul.values.tolist()
rust = plant_train.loc[ : , 'rust' ]
rust_val= rust.values.tolist()
scab = plant_train.loc[ : , 'scab' ]
scab_val= scab.values.tolist()
#img_name=[]
#health=[]
for i in range(0,(len(img_id)-1)):
  if mul_val[i]==1:
    shutil.copyfile(src_dir1 + img_name[i] + ".jpg", dst_dir1 + img_name[i] + ".jpg")
  elif rust_val[i]==1:
    shutil.copyfile(src_dir1 + img_name[i] + ".jpg", dst_dir2 + img_name[i] + ".jpg")
  elif scab_val[i]==1:
    shutil.copyfile(src_dir1 + img_name[i] + ".jpg", dst_dir3 + img_name[i] + ".jpg")
  else:
    shutil.copyfile(src_dir1 + img_name[i] + ".jpg", dst_dir4 + img_name[i] + ".jpg")

In [ ]:
import tensorflow as tf
import os
DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
      if(logs.get('acc')>=DESIRED_ACCURACY):
          print("Reached 99.9% accuracy so cancelling training!")
          self.model.stop_training=True

callbacks = myCallback()
    
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(), 
tf.keras.layers.Dense(128,activation=tf.nn.relu),
tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras import optimizers
adm=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='binary_crossentropy',optimizer=adm,metrics=['acc'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255) 
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/plant-pathology-2020-fgvc7/training/', 
    target_size=(150,150), 
    batch_size=80, 
    class_mode='binary'
)

filepath="/content/drive/My Drive/plant-pathology-2020-fgvc7/weights_ckpt.ckpt"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

history = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs=50,
    verbose=1,
    shuffle=True,
    callbacks=[callbacks, checkpoint]
)

print(history.history['acc'][-1])

KeyboardInterrupt: ignored

In [ ]:
model.save("/content/drive/My Drive/plant-pathology-2020-fgvc7/plant_pathology.h5")

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/plant-pathology-2020-fgvc7/plant_pathology.h5')

In [ ]:
file_list1 = os.listdir(src_dir1)

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is good")
  else:
    print(fn + " is bad")

Saving Test_0.jpg to Test_0 (2).jpg
Saving Test_1.jpg to Test_1 (1).jpg
Saving Test_2.jpg to Test_2 (1).jpg
Saving Test_3.jpg to Test_3 (1).jpg
Saving Test_4.jpg to Test_4 (1).jpg
[[0.]]
[0.]
Test_0.jpg is bad
[[0.]]
[0.]
Test_1.jpg is bad
[[0.]]
[0.]
Test_2.jpg is bad
[[1.]]
[1.]
Test_3.jpg is good
[[0.]]
[0.]
Test_4.jpg is bad


In [ ]:
import os, random
import shutil

src_dir1 = "/content/drive/My Drive/plant-pathology-2020-fgvc7/images/"

dst_dir1 = "/content/drive/My Drive/plant-pathology-2020-fgvc7/testing/"

#file_list1 = os.listdir(src_dir1)
#file_list2 = os.listdir(src_dir2)
img_id = plant_test.loc[ : , 'image_id' ]
img_name = img_id.values.tolist()
#img_name=[]
#health=[]
for i in range(0,(len(img_id)-1)):
  shutil.copyfile(src_dir1 + img_name[i] + ".jpg", dst_dir1 + img_name[i] + ".jpg")



In [ ]:
from csv import writer

In [ ]:
file_list1 = os.listdir(dst_dir1)
cls=[]

for f in file_list1:
  path = dst_dir1 + f
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  #print(classes)
  #print(classes[0])
  if classes[0]>0.5:
    cls.append(1)
  else:
    cls.append(0)



In [ ]:

col=['healthy']
for i in cls:
  col.append(i)

In [ ]:
print(col)

['healthy', 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1

In [ ]:
col[0]

'healthy'

In [ ]:
len(cls)

1820

In [ ]:
from csv import writer

In [ ]:
from csv import writer
from csv import reader

# Open the input_file in read mode and output_file in write mode
with open('/content/drive/My Drive/plant-pathology-2020-fgvc7/test.csv', 'r') as read_obj, \
        open('/content/drive/My Drive/plant-pathology-2020-fgvc7/test_1.csv', 'w', newline='') as write_obj:
    # Create a csv.reader object from the input file object
    csv_reader = reader(read_obj)
    # Create a csv.writer object from the output file object
    csv_writer = writer(write_obj)
    # Read each row of the input csv file as list
    i=0
    for row in csv_reader:
        # Append the default text in the row / list
        if i<=1820:
          row.append(col[i])
          i=i+1
        else:
          break
        # Add the updated row / list to the output file
        csv_writer.writerow(row)

In [ ]:
import tensorflow as tf
import os
DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
      if(logs.get('acc')>=DESIRED_ACCURACY):
          print("Reached 99.9% accuracy so cancelling training!")
          self.model.stop_training=True

callbacks = myCallback()
    
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Flatten(), 
tf.keras.layers.Dense(128,activation=tf.nn.relu),
tf.keras.layers.Dense(4, activation='softmax')
])

from tensorflow.keras import optimizers
adm=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='categorical_crossentropy',optimizer=adm,metrics=['acc'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255) 
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/plant-pathology-2020-fgvc7/training_2/', 
    target_size=(150,150), 
    batch_size=80, 
    class_mode='categorical'
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch=50,
    epochs=80,
    verbose=1,
    shuffle=True,
    callbacks=[callbacks]
)

print(history.history['acc'][-1])

Found 1820 images belonging to 4 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/80
50/50 [==============================] - 905s 18s/step - loss: 1.2410 - acc: 0.3970
Epoch 2/80
50/50 [==============================] - 124s 2s/step - loss: 1.0371 - acc: 0.5646
Epoch 3/80
50/50 [==============================] - 123s 2s/step - loss: 0.8124 - acc: 0.6773
Epoch 4/80
50/50 [==============================] - 122s 2s/step - loss: 0.5413 - acc: 0.8003
Epoch 5/80
50/50 [==============================] - 122s 2s/step - loss: 0.3070 - acc: 0.8914
Epoch 6/80
50/50 [==============================] - 122s 2s/step - loss: 0.1524 - acc: 0.9525
Epoch 7/80
50/50 [==============================] - 122s 2s/step - loss: 0.0610 - acc: 0.9886
Epoch 8/80
50/50 [==============================] - 122s 2s/step - loss: 0.0201 - acc: 0.9982
Epoch 9/80
50/50 [==============================] - 122s 2s/step - loss: 0.0101 - acc: 0.9992
0.9992424249649048


In [ ]:
model.save("/content/drive/My Drive/plant-pathology-2020-fgvc7/plant_pathology_diseases.h5")

In [ ]:
import tensorflow as tf


In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/plant-pathology-2020-fgvc7/plant_pathology_diseases.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255) 

test_generator = test_datagen.flow_from_directory(
    '/content/drive/My Drive/plant-pathology-2020-fgvc7/test_images/', 
    target_size=(150,150), 
    batch_size=80, 
    class_mode='categorical'
)


Found 1820 images belonging to 1 classes.


In [ ]:
pred_vals=model.predict(test_generator)

KeyboardInterrupt: ignored

In [ ]:
test_imgs = ["/content/drive/My Drive/plant-pathology-2020-fgvc7/test_images/testing/{}.jpg".format(x) for x in list(plant_test.image_id)]
test_set = pd.DataFrame( {'Images': test_imgs })

In [ ]:
from keras.preprocessing import image

Y_pred = []

for i in range(len(test_set)):
  img = image.load_img(path= test_set.Images[i],target_size=(150,150,3))
  img = image.img_to_array(img)
  test_img = img.reshape((1,150,150,3))
  img_class = model.predict_classes(test_img)
  prediction = img_class[0]
  Y_pred.append(prediction)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
  img = image.load_img(path="",target_size=(150,150,3))
  img = image.img_to_array(img)
  test_img = img.reshape((1,150,150,3))
  img_class = model.predict_classes(test_img)
  prediction = img_class[0]
  Y_pred.append(prediction)

In [ ]:
print(Y_pred)

[3, 1, 2, 3, 1, 3, 2, 2, 2, 1, 1, 0, 2, 2, 1, 3, 1, 2, 3, 1, 3, 3, 1, 3, 3, 3, 1, 1, 3, 1, 3, 1, 2, 2, 1, 2, 1, 3, 2, 1, 1, 1, 2, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 3, 2, 2, 1, 1, 2, 1, 0, 0, 2, 1, 3, 1, 1, 1, 2, 2, 3, 3, 3, 2, 3, 1, 2, 3, 1, 3, 1, 1, 2, 2, 1, 3, 3, 3, 1, 3, 2, 1, 2, 2, 1, 3, 2, 3, 3, 1, 1, 2, 2, 2, 3, 1, 2, 3, 2, 1, 1, 2, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 3, 3, 1, 1, 3, 1, 2, 2, 3, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 3, 0, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 3, 2, 3, 1, 0, 3, 2, 1, 1, 1, 3, 3, 1, 2, 2, 1, 1, 3, 0, 1, 1, 1, 3, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 3, 1, 2, 2, 1, 2, 2, 3, 1, 3, 1, 1, 1, 1, 1, 1, 3, 3, 2, 3, 3, 2, 2, 1, 1, 1, 3, 1, 2, 1, 1, 3, 2, 2, 3, 2, 2, 1, 2, 1, 1, 1, 1, 3, 3, 2, 3, 2, 2, 2, 3, 3, 2, 0, 2, 2, 2, 2, 2, 3, 1, 2, 3, 3, 3, 3, 1, 2, 3, 1, 3, 1, 3, 1, 3, 1, 1, 2, 0, 2, 2, 2, 2, 0, 1, 1, 3, 3, 3, 2, 2, 1, 1, 3, 3, 1, 1, 2, 2, 1, 1, 3, 2, 2, 1, 3, 1, 1, 3, 1, 1, 1, 1, 2, 2, 2, 2, 3, 1, 3, 1, 1, 2, 0, 1, 1, 2, 1, 2, 1, 1, 1, 2, 3, 

In [ ]:
len(Y_pred)

1820

In [ ]:
mul=["multiple"]
for i in  Y_pred:
  if i==0:
    mul.append(1)
  else:
    mul.append(0)

In [ ]:
print(mul)

['multiple_diseases', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

In [ ]:
mul[0]="multiple_diseases"

In [ ]:
rust=["rust"]
for i in  Y_pred:
  if i==1:
    rust.append(1)
  else:
    rust.append(0)

In [ ]:
scab=["scab"]
for i in  Y_pred:
  if i==2:
    scab.append(1)
  else:
    scab.append(0)

In [ ]:
from csv import writer
from csv import reader

# Open the input_file in read mode and output_file in write mode
with open('/content/drive/My Drive/plant-pathology-2020-fgvc7/test_3.csv', 'r') as read_obj, \
        open('/content/drive/My Drive/plant-pathology-2020-fgvc7/test_4.csv', 'w', newline='') as write_obj:
    # Create a csv.reader object from the input file object
    csv_reader = reader(read_obj)
    # Create a csv.writer object from the output file object
    csv_writer = writer(write_obj)
    # Read each row of the input csv file as list
    i=0
    for row in csv_reader:
        # Append the default text in the row / list
        if i<=1820:
          row.append(scab[i])
          i=i+1
        else:
          break
        # Add the updated row / list to the output file
        csv_writer.writerow(row)


In [ ]:
rust[1821]

IndexError: ignored